In [3]:
import re
from rnnmorph.predictor import RNNMorphPredictor
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Владислав\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
file = open('Capital_Karl_Marx.txt', 'r')
text = file.read()
text[0:500]

'Карл Генрих Маркс\n\nКапитал\n\n\n\nКнига первая: процесс производства капитала\n\n\n\nПредисловие\xa0к первому изданию\n\nТруд, первый том которого я предлагаю вниманию публики, составляет продолжение опубликованного в 1859 г. моего сочинения “К критике политической экономии”. Длительный перерыв между началом и продолжением вызван многолетней болезнью, которая все снова и снова прерывала мою работу.\n\nСодержание более раннего сочинения, упомянутого выше, резюмировано в первой главе этого тома. Я сделал это не '

In [18]:
words = word_tokenize(text)
words[0:500]

['Карл',
 'Генрих',
 'Маркс',
 'Капитал',
 'Книга',
 'первая',
 ':',
 'процесс',
 'производства',
 'капитала',
 'Предисловие',
 'к',
 'первому',
 'изданию',
 'Труд',
 ',',
 'первый',
 'том',
 'которого',
 'я',
 'предлагаю',
 'вниманию',
 'публики',
 ',',
 'составляет',
 'продолжение',
 'опубликованного',
 'в',
 '1859',
 'г.',
 'моего',
 'сочинения',
 '“',
 'К',
 'критике',
 'политической',
 'экономии',
 '”',
 '.',
 'Длительный',
 'перерыв',
 'между',
 'началом',
 'и',
 'продолжением',
 'вызван',
 'многолетней',
 'болезнью',
 ',',
 'которая',
 'все',
 'снова',
 'и',
 'снова',
 'прерывала',
 'мою',
 'работу',
 '.',
 'Содержание',
 'более',
 'раннего',
 'сочинения',
 ',',
 'упомянутого',
 'выше',
 ',',
 'резюмировано',
 'в',
 'первой',
 'главе',
 'этого',
 'тома',
 '.',
 'Я',
 'сделал',
 'это',
 'не',
 'только',
 'в',
 'интересах',
 'большей',
 'связности',
 'и',
 'полноты',
 'исследования',
 '.',
 'Самое',
 'изложение',
 'улучшено',
 '.',
 'Многие',
 'пункты',
 ',',
 'которые',
 'там',
 

In [16]:
predictor = RNNMorphPredictor(language='ru')

In [19]:
len(words)

491957

<h3>Получения свойств слов с RNN</h3>

Эта часть работает долго, можно перейти к следующей, результаты сохранены заранее

In [22]:
res = predictor.predict(words)

1/1 [==============================] - 793s 793s/step


In [32]:
nf = []
word = []
pos = []
tag = []
score = []
for i in res:
    nf.append(i.normal_form)
    word.append(i.word)
    pos.append(i.pos)
    tag.append(i.tag)
    score.append(i.score)
df = pd.DataFrame({'normal_form': nf, 'word': word, 'pos': pos, 'tag':tag, 'score':score})
df.head()

,normal_form,word,pos,tag,score
0,карла,Карл,NOUN,Case=Nom|Gender=Masc|Number=Sing,0.987951
1,генрих,Генрих,NOUN,Case=Nom|Gender=Masc|Number=Sing,0.986695
2,маркс,Маркс,NOUN,Case=Nom|Gender=Masc|Number=Sing,0.832564
3,капитал,Капитал,NOUN,Case=Nom|Gender=Masc|Number=Sing,0.954661
4,книга,Книга,NOUN,Case=Nom|Gender=Fem|Number=Sing,0.999114


In [33]:
df.to_csv('RNNwords.csv', index=False)

<h3>Поиск пар слов по предоставленным условиям</h3>

In [4]:
df = pd.read_csv('RNNwords.csv')
df.head()

,normal_form,word,pos,tag,score
0,карла,Карл,NOUN,Case=Nom|Gender=Masc|Number=Sing,0.987951
1,генрих,Генрих,NOUN,Case=Nom|Gender=Masc|Number=Sing,0.986695
2,маркс,Маркс,NOUN,Case=Nom|Gender=Masc|Number=Sing,0.832564
3,капитал,Капитал,NOUN,Case=Nom|Gender=Masc|Number=Sing,0.954661
4,книга,Книга,NOUN,Case=Nom|Gender=Fem|Number=Sing,0.999114


In [5]:
def get_tags_from_str(tag_str):
    params = tag_str.split('|')
    tags = {}
    if len(params)>1:
        for i in params:
            tag = i.split('=')
            tags[tag[0]] = tag[1]
    return tags

In [13]:
%%time
result = []
left_word = { 'word': '', 'POS': '', 'gender': '', 'number': '', 'case': '', 'idx':-2 }

for i in df.index:
    if i%int(df.shape[0]/100)==0: print(str(i//int(df.shape[0]/100))+'%')
    #m = morphy.parse(words[i])[0]
    tag = get_tags_from_str(df.iloc[i].tag)
    if df.iloc[i].pos in ['NOUN', 'ADJ'] and len(set(tag.keys()).intersection(set(['Gender', 'Case', 'Number'])))==3:
        if (i-left_word['idx']==1 and left_word['gender']==tag['Gender'] and
            left_word['number']==tag['Number'] and left_word['case']==tag['Case'] and
            left_word['POS']!=df.iloc[i].pos):
            result.append(left_word['word'] + ' ' + df.iloc[i].normal_form)
        left_word['idx'] = i
        left_word['word'] = df.iloc[i].normal_form
        left_word['POS'] = df.iloc[i].pos
        left_word['gender'] = tag['Gender']
        left_word['number'] = tag['Number']
        left_word['case'] = tag['Case']

0%
1%
2%
3%
4%
5%
6%
7%
8%
9%
10%
11%
12%
13%
14%
15%
16%
17%
18%
19%
20%
21%
22%
23%
24%
25%
26%
27%
28%
29%
30%
31%
32%
33%
34%
35%
36%
37%
38%
39%
40%
41%
42%
43%
44%
45%
46%
47%
48%
49%
50%
51%
52%
53%
54%
55%
56%
57%
58%
59%
60%
61%
62%
63%
64%
65%
66%
67%
68%
69%
70%
71%
72%
73%
74%
75%
76%
77%
78%
79%
80%
81%
82%
83%
84%
85%
86%
87%
88%
89%
90%
91%
92%
93%
94%
95%
96%
97%
98%
99%
100%
CPU times: total: 1min 46s
Wall time: 1min 46s


In [14]:
len(result)

27403

In [15]:
# Drop duplicates
result1 = list(set(result))
len(result1)

7520

In [17]:
file = open('task1_results.txt', 'r')
task1_result = file.read().split('\n')
task1_result

['настоящий бедствие',
 'небольшой объём',
 'особенный форма',
 'возвращение свой',
 'наследственный привилегия',
 'этот половина',
 'этот нивелировка',
 'настоящий цель',
 'недостаточный понимание',
 'высокий смертность',
 'свой тонус',
 'парламентский отчёт',
 'действительный отдых',
 'адекватный бытие',
 'денежный маска',
 'иной отрасль',
 'увеличение рабочий',
 'какой результат',
 'малый остановка',
 'непрерывный кругооборот',
 'функциональный форма',
 'английский канцлер',
 'свободный труд',
 'буржуазный право',
 'общественный обмен',
 'узловой пункт',
 'широкий принцип',
 'свой накопление',
 'самый капитал',
 'идеальный образ',
 'этот тетрадь',
 'разительный контраст',
 'существенный основа',
 'разработка буржуазный',
 'глубокий обоснование',
 'нелепый формула',
 'весь производство',
 'случайный проявление',
 'спорадический применение',
 'хороший способ',
 'простой образование',
 'элемент товарный',
 'реализация такой',
 'какой-то операция',
 'сливочный масло',
 'ный капитал',
 '

In [31]:
res1_task1_diff = list(set(result1).difference(set(task1_result)))
res1_task1_diff.sort()
res1_task1_diff

["'тый процесс",
 '.рабочий период',
 '000с подразделение',
 '1-й издание',
 '1-й миллион',
 '1-й продажа',
 '10-летний возраст',
 '10-часовой продолжительность',
 '101/2-часовой труд',
 '12-летний возраст',
 '12-часовой труд',
 '13-летний возраст',
 '13–14-часовой труд',
 '15-й неделя',
 '15-летний возраст',
 '15—55-летний возраст',
 '16-летний возраст',
 '18-й неделя',
 '18-летний возраст',
 '20-летний возраст',
 '21-й неделя',
 '23-й год',
 '24-часовой процесс',
 '30-й день',
 '30-летний возраст',
 '30-часовой работа',
 '37-й неделя',
 '4-й неделя',
 '4-й раздел',
 '4-летний ребёнок',
 '40-дюймовый сукно',
 '42-летний возраст',
 '51-й неделя',
 '61/2-часовой труд',
 '8—12-летнсй возраст',
 '9-летний возраст',
 '9-летний промежуток',
 '9—13-летний возраст',
 'beaux yeux',
 "chandler 's",
 'faux frais',
 'i-й период',
 'ii подразделение',
 'iii-ия период',
 'm. догма',
 'nexus rerum',
 'oбычный уверение',
 'reports etc',
 'xvi век',
 'xvii столетие',
 'xviii век',
 'xviii столетие',
 

In [32]:
len(res1_task1_diff)

3179

<h3>Свойства RNN</h3>

- больше правильно определённых тегов на основе контекста
- понимаются сокращения, слова по смыслу схожие с прилагательными
- числительные принимаются за прилагательные
- понимает зашумлённые слова. Например: ру-ка, часто из-за этого возникают ошибки

In [33]:
task1_res1_diff = list(set(task1_result).difference(set(result1)))
task1_res1_diff.sort()
task1_res1_diff

['-переменный капитал',
 '18-часовой день',
 'авансирование постоянный',
 'акт этот',
 'английский рабочий',
 'большинство свой',
 'бравый мозес',
 'ваш данные',
 'ваш детство',
 'ваш дух',
 'ваш идея',
 'ваш сведение',
 'ваш случай',
 'ваш утверждение',
 'введение новый',
 'ведение весь',
 'величина каждый',
 'величина сам',
 'величина свой',
 'величина тот',
 'величина этот',
 'весь англия',
 'весь буржуазия',
 'весь величина',
 'весь вероятность',
 'весь вес',
 'весь время',
 'весь год',
 'весь государство',
 'весь движение',
 'весь день',
 'весь добыча',
 'весь дробь',
 'весь дюжина',
 'весь европа',
 'весь законодательство',
 'весь затрата',
 'весь здание',
 'весь исследование',
 'весь история',
 'весь книга',
 'весь количество',
 'весь комичность',
 'весь конструкция',
 'весь линия',
 'весь масса',
 'весь машина',
 'весь мудрость',
 'весь население',
 'весь нация',
 'весь неделя',
 'весь нелепость',
 'весь ночь',
 'весь обращение',
 'весь общество',
 'весь операция',
 'весь основ

In [34]:
len(task1_res1_diff)

1480

<h3>Свойства PyMorphy</h3>

- местоимения выдаются за прилагательные

In [28]:
intersection = list(set(result1).intersection(set(task1_result)))
intersection.sort()
intersection

['.больший часть',
 '/необходимый труд',
 '10-й неделя',
 '12-й неделя',
 '12-часовой день',
 '13-й неделя',
 '14-й неделя',
 '15-часовой период',
 '16-й неделя',
 '19-й неделя',
 '2-й неделя',
 '20-летний модистка',
 '220-футовый скорость',
 '24-часовой система',
 '3-й неделя',
 '5-й неделя',
 '5-футовый ход',
 '6-й неделя',
 '7-й неделя',
 '8-фунтовой ткань',
 '9-й неделя',
 'cобственный смысл',
 'iii-й случай',
 'абсолютный величина',
 'абсолютный граница',
 'абсолютный итог',
 'абсолютный количество',
 'абсолютный масса',
 'абсолютный обогащение',
 'абсолютный правило',
 'абсолютный предел',
 'абсолютный преобладание',
 'абсолютный противоречие',
 'абсолютный скорость',
 'абсолютный стоимость',
 'абсолютный товар',
 'абсолютный увеличение',
 'абсолютный уменьшение',
 'абсолютный форма',
 'абстрактный богатство',
 'абстрактный мышление',
 'абстрактный определённость',
 'абстрактный стоимость',
 'абстракция разумный',
 'авансированный капитал',
 'авансированный капиталь',
 'авансиров

In [30]:
len(intersection)

4341

<h3>Объединение</h3>

- При объединении результаты получаются чище